In [33]:
%matplotlib qt

import numpy as np
import hyperspy.api as hs
import matplotlib.pyplot as plt
import cv2

In [147]:
# Tiff files

# images_1
# data_file_dir = "/media/ivan/Ivan/jad/images_1/plastic/src_Specim-FX17e-076900055547_00.tiff"
# data_file_dir = "/media/ivan/Ivan/jad/images_1/plastic/src_Specim-FX17e-076900055547_01.tiff"
# data_file_dir = "/media/ivan/Ivan/jad/images_1/plastic/src_Specim-FX17e-076900055547_02.tiff"
# data_file_dir = "/media/ivan/Ivan/jad/images_1/plastic/src_Specim-FX17e-076900055547_03.tiff"

# images_2
# data_file_dir = "/media/ivan/Ivan/jad/images_2/normal/src_Specim-FX17e-076900055547_00.tiff"
# data_file_dir = "/media/ivan/Ivan/jad/images_2/src_Specim-FX17e-076900055547_01.tiff"               # bad image
# data_file_dir = "/media/ivan/Ivan/jad/images_2/normal/src_Specim-FX17e-076900055547_02.tiff"

# images_3
# data_file_dir = "/media/ivan/Ivan/jad/images_3/normal/src_Specim-FX17e-076900055547_03.tiff"
# data_file_dir = "/media/ivan/Ivan/jad/images_3/normal/src_Specim-FX17e-076900055547_11.tiff"
# data_file_dir = "/media/ivan/Ivan/jad/images_3/normal/src_Specim-FX17e-076900055547_21.tiff"
# data_file_dir = "/media/ivan/Ivan/jad/images_3/normal/src_Specim-FX17e-076900055547_27.tiff"
# data_file_dir = "/media/ivan/Ivan/jad/images_3/normal/src_Specim-FX17e-076900055547_32.tiff"
# data_file_dir = "/media/ivan/Ivan/jad/images_3/normal/src_Specim-FX17e-076900055547_37.tiff"
# data_file_dir = "/media/ivan/Ivan/jad/images_3/normal/src_Specim-FX17e-076900055547_43.tiff"
# data_file_dir = "/media/ivan/Ivan/jad/images_3/normal/src_Specim-FX17e-076900055547_51.tiff"
# data_file_dir = "/media/ivan/Ivan/jad/images_3/normal/src_Specim-FX17e-076900055547_54.tiff"
# data_file_dir = "/media/ivan/Ivan/jad/images_3/normal/src_Specim-FX17e-076900055547_69.tiff"
# data_file_dir = "/media/ivan/Ivan/jad/images_3/normal/src_Specim-FX17e-076900055547_71.tiff"

# images_4
# data_file_dir = "/media/ivan/Ivan/jad/images_4/plastic/src_Specim-FX17e-076900055547_00.tiff"
data_file_dir = "/media/ivan/Ivan/jad/images_4/plastic/src_Specim-FX17e-076900055547_01.tiff"       # Plastic yes
# data_file_dir = "/media/ivan/Ivan/jad/images_4/plastic/src_Specim-FX17e-076900055547_02.tiff"
# data_file_dir = "/media/ivan/Ivan/jad/images_4/plastic/src_Specim-FX17e-076900055547_03.tiff"
# data_file_dir = "/media/ivan/Ivan/jad/images_4/plastic/src_Specim-FX17e-076900055547_04.tiff"       # plastic YES
# data_file_dir = "/media/ivan/Ivan/jad/images_4/plastic/src_Specim-FX17e-076900055547_05.tiff"

example_image = []

_, example_image = cv2.imreadmulti(
    filename=data_file_dir,
    mats=example_image,
    flags=cv2.IMREAD_UNCHANGED
)

example_image = np.asarray(example_image)
example_image = np.transpose(example_image, (0, 2, 1))
# print(example_image.shape)


In [35]:
label_correspondences = {
    "meat": 0,
    "fat": 1,
    "conveyor": 2,
    "plastic": 3
}

# Add label correspondences, change integer values to change heat map
color_correspondences = {
    label_correspondences["meat"]: "g",
    label_correspondences["fat"]: "w",
    label_correspondences["conveyor"]: "b",
    label_correspondences["plastic"]: "r",
}

In [36]:
# Initialize segmented image
segmented_image = np.zeros(example_image.shape[:2], dtype=np.uint8)

# Conveyor classification
conveyor_pixels = np.where((example_image[:, :, 30] - example_image[:, :, 25]) > 0.0)
segmented_image[conveyor_pixels[0], conveyor_pixels[1]] = 2

In [37]:
# Fat classification
fat_condition = ((example_image[:, :, 80] - example_image[:, :, 61])/(80 - 61) - (example_image[:, :, 61] - example_image[:, :, 53])/(61 - 53))
extra_condition = ((example_image[:, :, 154] - example_image[:, :, 112])/(154 - 112))
fat_pixels = np.where(
    np.logical_and(
        np.logical_and(
            fat_condition > -0.0025,
            (example_image[:, :, 30] - example_image[:, :, 25]) < 0.0
        ),
        (example_image[:, :, 61] - example_image[:, :, 57]) > 0.00045
    )
)

segmented_image[fat_pixels[0], fat_pixels[1]] = 1

In [119]:
# # Plastic classification
# second_condition = (example_image[:, :, 61] + example_image[:, :, 57]) > 0.05
# first_condition = (example_image[:, :, 61] - example_image[:, :, 57]) < 0.00045 # 0.00045 -> my value || 0.0015 -> jad value

# # slope1 = (example_image[:, :, 61]-example_image[:, :, 57])/4 
# # slope2 = (example_image[:, :, 80]-example_image[:, :, 61])/19
# # third_condition = (np.abs(slope2) - np.abs(slope1)) > 1

# plastic_pixels = np.where(np.logical_and(first_condition, second_condition))
# segmented_image[plastic_pixels[0], plastic_pixels[1]] = 3

# Plastic classification
light_check = (example_image[:, :, 61] + example_image[:, :, 57]) > 0.05
red_check = (example_image[:, :, 61] - example_image[:, :, 57]) < 0.0015      # between 0.0006 and 0.0005 the value disappears --> my value 0.00045
crease_check = (example_image[:, :, 153] - example_image[:, :, 150]) < 0.0
extra_check = (example_image[:, :, 60] - example_image[:, :, 53]) > 0.001

values = [example_image[:, :, 57], example_image[:, :, 58], example_image[:, :, 59], example_image[:, :, 60], example_image[:, :, 61]]
test = sum(values)/len(values)

new_check = (max(values) - test) < 0.042

extra_check = False
plastic_pixels = np.where(
    np.logical_and(
        np.logical_and(red_check, light_check),
        np.logical_or(crease_check, extra_check)
    )    
)
segmented_image[plastic_pixels[0], plastic_pixels[1]] = 3

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [181]:
# ========================================= kill pixels =========================================

threshold = 0.32

new_plot = np.ones_like(example_image[:, :, 1])

for x in range(example_image.shape[0]):
    for y in range(example_image.shape[1]):
        # mean_value = np.mean(example_image[x, y, :])
        values = [example_image[x, y, 57], example_image[x, y, 58], example_image[x, y, 59], example_image[x, y, 60], example_image[x, y, 61]]
        test = sum(values)/len(values)
        slope = (example_image[x, y, 58] - example_image[x, y, 60])/3
        

        if x == 193 and y == 572:
            print(test)
            print(max(values))
            print(max(values)-test)
            print(slope)

        # if (max(values) - test) > 0.0015 and (example_image[x, y, 61] - example_image[x, y, 57]) < 0.0035:    # 0.00315 and 0.00415
        #     print(max(values)-test)
        #     print(x,y)
        #     new_plot[x, y] = 0
            
        if example_image[x, y, 60] - example_image[x, y, 58] < 0.000006  and ((max(values)-test) > 0.0000033): # slope < 0 and 
            
            print(example_image[x, y, 60] - example_image[x, y, 58])
            print(max(values)-test)
            print(x,y)
            new_plot[x, y] = 0

plt.imshow(new_plot, cmap='gray')
plt.show()

# hyper_data = hs.signals.Signal1D(example_image)
# print(hyper_data)
# hyper_data.plot()

0.37052834033966064
0.41261223
0.04208388924598694
-0.014914105335871378
-1.180172e-05
0.00480532944202422
267 244
-0.00019162893
0.005163103342056274
267 245
-0.0001367107
0.004427404701709742
267 246
-9.1575086e-05
0.004916806519031522
267 248
-0.0003175512
0.004361973702907557
268 248
-0.00039175898
0.0046671301126480075
268 250
-0.00026848167
0.005079840123653409
268 251
-0.0003034547
0.004617948830127713
269 248
-0.0006631091
0.00442510396242142
269 249
-0.0008540824
0.0034482479095458984
269 250
-0.00050307065
0.004420061409473416
269 251
-0.0001796037
0.004586783051490795
270 237
-0.0005726516
0.0056108832359313965
270 238
-3.8042665e-05
0.004992508888244623
270 239
-4.798174e-05
0.005402117967605591
270 248
-0.00013298541
0.0044080972671508845
270 250
-0.00040084124
0.005585438013076788
271 236
-0.00018778443
0.004089823365211492
271 237
-0.00034192204
0.004944711923599243
271 238
-0.00028133392
0.004228910803794866
271 239
-0.00044238567
0.005538693070411688
271 241
-0.0001939

In [ ]:
# Visualization
color_map = colors.ListedColormap([
    color_correspondences[0],
    color_correspondences[1],
    color_correspondences[2],
    color_correspondences[3]
])
print([
    color_correspondences[0],
    color_correspondences[1],
    color_correspondences[2],
    color_correspondences[3]
])

plt.imshow(segmented_image, interpolation="nearest", cmap=color_map, vmin=0, vmax=4)
plt.show()